# Benders

In [ ]:
import docplex.mp
from docplex.mp.model import Model
import cplex
from cplex.exceptions import CplexError
import math

## Sketch / Mk. 0

In [10]:
def read_instance(file_path):
    """
    Reads a CVRP instance from a file and returns the node coordinates and demands.
    """
    node_coords = {}
    demands = {}
    capacity = 0
    with open(file_path, 'r') as file:
        section = None
        for line in file:
            line = line.strip()
            if line.startswith('CAPACITY'):
                capacity = int(line.split(':')[1].strip())
            elif line.startswith('NODE_COORD_SECTION'):
                section = 'NODE_COORD_SECTION'
            elif line.startswith('DEMAND_SECTION'):
                section = 'DEMAND_SECTION'
            elif line.startswith('DEPOT_SECTION'):
                break
            elif section == 'NODE_COORD_SECTION' and line != '':
                parts = line.split()
                node_id = int(parts[0])
                x_coord = float(parts[1])
                y_coord = float(parts[2])
                node_coords[node_id] = (x_coord, y_coord)
            elif section == 'DEMAND_SECTION' and line != '':
                parts = line.split()
                node_id = int(parts[0])
                demand = float(parts[1])
                demands[node_id] = demand
    return node_coords, demands, capacity

def compute_distance_matrix(node_coords):
    """
    Computes the Euclidean distance matrix between all nodes.
    """
    nodes = list(node_coords.keys())
    distance_matrix = {}
    for i in nodes:
        distance_matrix[i] = {}
        for j in nodes:
            xi, yi = node_coords[i]
            xj, yj = node_coords[j]
            distance = math.hypot(xi - xj, yi - yj)
            distance_matrix[i][j] = distance
    return distance_matrix

In [ ]:
from docplex.mp.model import Model
import math

def read_instance(file_path):
    """
    Reads a CVRP instance from a file and returns the node coordinates, demands, and capacity.
    """
    node_coords = {}
    customers = []
    depot_coords = tuple()
    demands = {}
    capacity = 0
    with open(file_path, 'r') as file:
        section = None
        for line in file:
            line = line.strip()
            if line.startswith('CAPACITY'):
                capacity = int(line.split(':')[1].strip())
            elif line.startswith('NODE_COORD_SECTION'):
                section = 'NODE_COORD_SECTION'
            elif line.startswith('DEMAND_SECTION'):
                section = 'DEMAND_SECTION'
            elif line.startswith('DEPOT_SECTION'):
                break
            elif section == 'NODE_COORD_SECTION' and line != '':
                parts = line.split()

                node_id = int(parts[0])
                x_coord = int(parts[1])
                y_coord = int(parts[2])
                node_coords[node_id] = (x_coord, y_coord)

                if node_id == 0:
                    depot_coords = node_coords[node_id]
                else:
                    customers.append({
                        'number': node_id,
                        'x': x_coord,
                        'y': y_coord,
                        'demand': 0
                    })
            elif section == 'DEMAND_SECTION' and line != '':
                parts = line.split()

                node_id = int(parts[0])
                demand = float(parts[1])
                demands[node_id] = demand

                if node_id != 0:
                    customers[node_id]['demand'] = demand
    return node_coords, demands, capacity

def compute_distance_matrix(node_coords):
    """
    Computes the Euclidean distance matrix between all nodes.
    """
    nodes = list(node_coords.keys())
    distance_matrix = {}
    for i in nodes:
        distance_matrix[i] = {}
        xi, yi = node_coords[i]
        for j in nodes:
            xj, yj = node_coords[j]
            distance = math.hypot(xi - xj, yi - yj)
            distance_matrix[i][j] = distance
    return distance_matrix

def solve_cvrp_benders(node_coords, demands, capacity, max_vehicles):
    """
    Solves the CVRP using Benders Decomposition with flow constraints.
    """
    # Initialize variables
    nodes = list(node_coords.keys())
    customers = nodes.copy()
    customers.remove(0)  # Remove depot

    arcs = [(i, j) for i in nodes for j in nodes if i != j]

    distance_matrix = compute_distance_matrix(node_coords)
    
    # Master problem model
    master = Model("CVRP_Master")

    # Decision variables x_ij
    x_vars = {}
    for i, j in arcs:
        x_vars[i, j] = master.binary_var(name="x_{0}_{1}".format(i, j))

    # Objective function: Minimize total distance
    master.minimize(master.sum(distance_matrix[i][j] * x_vars[i, j] for i, j in arcs))

    # Constraints:

    # Each customer is visited exactly once
    for k in customers:
        master.add_constraint(master.sum(x_vars[i, k] for i in nodes if i != k) == 1, "inflow_{0}".format(k))
        master.add_constraint(master.sum(x_vars[k, j] for j in nodes if j != k) == 1, "outflow_{0}".format(k))

    # Vehicle limit constraint
    master.add_constraint(master.sum(x_vars[0, j] for j in nodes if j != 0) <= max_vehicles, "vehicle_limit")

    # Subproblem cuts will be added iteratively
    iteration = 0
    max_iterations = 100
    epsilon = 1e-6  # Tolerance for capacity violation

    while True:
        iteration += 1
        print("Iteration:", iteration)

        # Solve the master problem
        solution = master.solve(log_output=False)

        if solution is None:
            print("No feasible solution found in the master problem.")
            break

        # Extract x_ij values
        x_values = {(i, j): x_vars[i, j].solution_value for i, j in arcs}

        # Build the solution graph
        solution_arcs = [(i, j) for (i, j) in arcs if x_values[i, j] > 0.5]

        # Extract routes from the solution
        routes = extract_routes(solution_arcs, 0)

        # Check capacity feasibility
        violated_routes = []
        for route in routes:
            route_demand = sum(demands.get(node, 0) for node in route if node != 0)
            if route_demand > capacity + epsilon:
                violated_routes.append((route, route_demand))

        if not violated_routes:
            print("Optimal solution found.")
            for idx, route in enumerate(routes):
                print("Route {0}: {1}, Load: {2}".format(idx + 1, route, sum(demands.get(node, 0) for node in route)))
            break  # Feasible solution found

        # Add Benders cuts for each violated route
        for route, load in violated_routes:
            # Create a set of nodes in the route excluding the depot
            S = set(route)
            S.discard(0)

            # Create the cut: sum_{i in S, j not in S} x_ij >= 1
            cut_expr = master.sum(x_vars[i, j] for i in S for j in nodes if j not in S)
            master.add_constraint(cut_expr >= 1, "Benders_cut_{0}_{1}".format(iteration, routes.index(route)))

        # Check iteration limit
        if iteration >= max_iterations:
            print("Reached maximum number of iterations without finding a feasible solution.")
            break

def extract_routes(solution_arcs, depot):
    """
    Extracts routes from the solution arcs.
    """
    routes = []
    unvisited_arcs = solution_arcs.copy()
    while unvisited_arcs:
        route = [depot]
        current_node = depot
        while True:
            next_arc = [arc for arc in unvisited_arcs if arc[0] == current_node]
            if not next_arc:
                break
            next_node = next_arc[0][1]
            route.append(next_node)
            unvisited_arcs.remove(next_arc[0])
            current_node = next_node
            if current_node == depot:
                break
        if route[0] == depot and route[-1] == depot:
            routes.append(route)
        else:
            # Incomplete route, possible subtour
            routes.append(route)
    return routes

# Example usage
if __name__ == "__main__":
    file_path = "../Instances/small_instance_19.txt"  # Replace with your instance file path
    node_coords, demands, capacity = read_instance(file_path)
    max_vehicles = 5  # Adjust based on your problem
    solve_cvrp_benders(node_coords, demands, capacity, max_vehicles)


Iteration: 1


KeyboardInterrupt: 